In [103]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from pymongo import MongoClient

In [104]:
# Import NYT best sellers from 2011 to 2018.
# Source: https://data.world/typhon/new-york-times-bestsellers-from-2011-to-2018/workspace/file?filename=books_uniq_weeks.csv
nyt_bestsellers = "books_uniq_weeks.csv"
nyt_df = pd.read_csv(nyt_bestsellers)

# Shortening to make sure it runs withoua problem or taking all night.
nyt_df = nyt_df.head(50)


In [105]:
# Goodreads example URL: 
#https://www.goodreads.com/search?q=1594634025

nyt_df['goodreads_url'] = 'https://www.goodreads.com/search?q=' + nyt_df['primary_isbn10']
nyt_df.head()

,publisher,dagger,book_review_link,author,primary_isbn10,price,primary_isbn13,sunday_review_link,date,first_chapter_link,contributor,title,age_group,weeks_on_list,goodreads_url
0,Riverhead,0,https://www.nytimes.com/2015/01/05/books/the-g...,Paula Hawkins,1594634025,0,9.78159E+12,https://www.nytimes.com/2015/02/01/books/revie...,2/19/17,NaN,by Paula Hawkins,THE GIRL ON THE TRAIN,NaN,102,https://www.goodreads.com/search?q=1594634025
1,Scribner,0,NaN,Anthony Doerr,1501173219,0,9.7815E+12,https://www.nytimes.com/2014/05/11/books/revie...,5/7/17,NaN,by Anthony Doerr,ALL THE LIGHT WE CANNOT SEE,NaN,81,https://www.goodreads.com/search?q=1501173219
2,Vintage,0,NaN,E L James,525431888,0,9.78053E+12,NaN,3/5/17,NaN,by E. L. James,FIFTY SHADES DARKER,NaN,66,https://www.goodreads.com/search?q=525431888
3,St. Martin's,0,NaN,Kristin Hannah,1466850604,0,9.78147E+12,NaN,10/29/17,NaN,by Kristin Hannah,THE NIGHTINGALE,NaN,63,https://www.goodreads.com/search?q=1466850604
4,Penguin Group,0,https://www.nytimes.com/2009/02/19/books/19mas...,Kathryn Stockett,1440697663,0,9.78144E+12,NaN,4/8/12,NaN,by Kathryn Stockett,THE HELP,NaN,58,https://www.goodreads.com/search?q=1440697663


In [106]:
# Get soup object.

# resp = requests.get("https://www.goodreads.com/search?q=1594634025")
# soup = BeautifulSoup(resp.content, "html.parser")
# print(soup)

def get_html(isbn):
    url = "https://www.goodreads.com/search?q={}".format(isbn)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    return soup

nyt_df['soup_object'] = nyt_df['primary_isbn10'].map(lambda x: get_html(x))

nyt_df.head()

,publisher,dagger,book_review_link,author,primary_isbn10,price,primary_isbn13,sunday_review_link,date,first_chapter_link,contributor,title,age_group,weeks_on_list,goodreads_url,soup_object
0,Riverhead,0,https://www.nytimes.com/2015/01/05/books/the-g...,Paula Hawkins,1594634025,0,9.78159E+12,https://www.nytimes.com/2015/02/01/books/revie...,2/19/17,NaN,by Paula Hawkins,THE GIRL ON THE TRAIN,NaN,102,https://www.goodreads.com/search?q=1594634025,"<!DOCTYPE html> <html class=""desktop ""> <head..."
1,Scribner,0,NaN,Anthony Doerr,1501173219,0,9.7815E+12,https://www.nytimes.com/2014/05/11/books/revie...,5/7/17,NaN,by Anthony Doerr,ALL THE LIGHT WE CANNOT SEE,NaN,81,https://www.goodreads.com/search?q=1501173219,"<!DOCTYPE html> <html class=""desktop ""> <head..."
2,Vintage,0,NaN,E L James,525431888,0,9.78053E+12,NaN,3/5/17,NaN,by E. L. James,FIFTY SHADES DARKER,NaN,66,https://www.goodreads.com/search?q=525431888,"<!DOCTYPE html> <html class=""desktop ""> <head..."
3,St. Martin's,0,NaN,Kristin Hannah,1466850604,0,9.78147E+12,NaN,10/29/17,NaN,by Kristin Hannah,THE NIGHTINGALE,NaN,63,https://www.goodreads.com/search?q=1466850604,"<!DOCTYPE html> <html class=""desktop ""> <head..."
4,Penguin Group,0,https://www.nytimes.com/2009/02/19/books/19mas...,Kathryn Stockett,1440697663,0,9.78144E+12,NaN,4/8/12,NaN,by Kathryn Stockett,THE HELP,NaN,58,https://www.goodreads.com/search?q=1440697663,"<!DOCTYPE html> <html class=""desktop ""> <head..."


In [107]:
# Get rating.

def get_rating(soup_object):
    try:
        rating = soup_object.find("span", attrs={"itemprop": "ratingValue"})
        rating = rating.get_text()
        rating = rating.encode("utf-8")
        rating = rating.replace('\n', ' ')
        rating = float(rating)
        return rating
    except:
        return "not found on GoodReads"
    
nyt_df['rating'] = nyt_df['soup_object'].map(lambda x: get_rating(x))

In [108]:
# Get number of ratings.

def get_rating_count(soup_object):
    try:
        rating_count = soup_object.find("meta", attrs={"itemprop": "ratingCount"})
        rating_count = rating_count.get_text()
        rating_count = rating_count.replace('ratings', '').replace(',', '').strip()
        rating_count = int(rating_count)
        return rating_count
    except:
        return "not found on GoodReads"
    
nyt_df['rating_count'] = nyt_df['soup_object'].map(lambda x: get_rating_count(x))

In [109]:
# Get number of reviews.

def get_review_count(soup_object):
    try:
        reviews = soup_object.find("meta", attrs={"itemprop": "reviewCount"})
        reviews = reviews.get_text()
        reviews = reviews.replace('reviews', '').replace(',', '').strip()
        reviews = int(reviews)
        return reviews
    except:
        return "not found on GoodReads"
    
nyt_df['review_count'] = nyt_df['soup_object'].map(lambda x: get_review_count(x))

In [119]:
# Get related / suggested title in right rail.

def get_book_carousel(soup_object):
    try:
        temp_list = []
        book_carousel = soup_object.find_all("li", {"class":"cover"})
                                             
        for i in range(len(book_carousel)):
            title = soup_object[i].find("a").find("img")['alt'].encode('utf-8')
            temp_list.append(title)
        
        if temp_list == []:
            return 'no suggested titles'
        else:
            return temp_list
    except:
        return "not found on GoodReads"

nyt_df['related_titles'] = nyt_df['soup_object'].map(lambda x: get_book_carousel(x))


In [124]:
# Create a list in which we'll append each title's dictionary to add to Mongo.

mongo = []

for i in range(len(nyt_df)):
    temp_dict= { 
        "isbn": nyt_df.iloc[i]['primary_isbn10'],
        "publisher": nyt_df.iloc[i]['publisher'],
        "author": nyt_df.iloc[i]['author'],
        "title": nyt_df.iloc[i]['title'],
        "weeks_on_list": nyt_df.iloc[i]['weeks_on_list'],
        "goodreads_url": nyt_df.iloc[i]['goodreads_url'],
        "goodreads_html": str(nyt_df.iloc[i]['soup_object']),
        "related_titles": nyt_df.iloc[i]['related_titles']}
    mongo.append(temp_dict)

In [125]:
# Establish a connection to Mongo and insert the data.

client = MongoClient()
with client:
    db = client.nyt_bestsellers
    db.bestsellers.insert_many(mongo)